<a href="https://colab.research.google.com/github/calamistratus/Doc_to_csv_parser/blob/main/doc-to-csv-parser-COLAB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import os
import numpy as np
import pandas as pd

from pypdf import PdfReader
from tqdm import tqdm

import doc2pdf

In [ ]:
API_KEY = input('Input your API key: ')

url = 'https://api.deepseek.com/chat/completions'

headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {API_KEY}'
}

baseline_promt = '''CREATE TABLE resumes (
first_name VARCHAR, # Always in English
second_name VARCHAR, # Always in English
third_name VARCHAR, # Always in English
gender CHAR, # m/f/o(for other)
citizenship CHAR, # Always 2 English letters (for example RU or KZ)
relocation_willingness SMALLINT, #0-not_ready, 1-ready, 2-other
phone BIGINT,
email VARCHAR,
github VARHCAR, # Github link
tg VARCHAR, # A telegram username, example: '@pavel123'
age SMALLINT,
languages JSONB, # A1-1, A2-2, B1-3, B2-4, C1-5, C2-6, NATIVE-6, example: '{"EN":6, "RU":1}'
stack JSONB, # Candidates skills, example: '["scikit-learn", "sql"]'
experience REAL, # Amount of years of expirience, for example: 2 years and 6 months: 2.5
desired_position VARCHAR, # Data Scientist, for example
city VARCHAR, # In English, lowercase
wanted_salary INT, # Parse this from resume
estimated_salary INT, # Your take on what candidate's salary should be
education_level VARCHAR, # Few words, PhD or Master's degree and etc., in English
education_analysis VARCHAR, # List the candidate's education, courses, briefly analyze it
experience_analysis VARCHAR, # Your expirience analysys, companies the candidate has worked in, pet projects
salary_analysis VARCHAR, # Your salary analysis, why have you chosen this amount
overall_analysis VARCHAR, # Your overall analysis
contacted SMALLINT # Always 0
)

Parse the following files into python dict within the format detailed above. Write no comments'''

In [ ]:
def call(promt):
    data = {
    'model': 'deepseek-chat',
    'messages': [
        {'role': 'system', 'content': 'You are a professional assistant'},
        {'role': 'user', 'content': promt if type(promt) is str else ' '.join(promt)}
    ],
    'stream': False,
    'temperature':.4}

    response = requests.post(url, headers=headers, json=data)

    if response.status_code == 200:
        result = response.json()
        return result['choices'][0]['message']['content']
    else:
        print('Request failed, error code:', response.status_code)

In [ ]:
def get_text(file_name):
    doc2pdf.convert(file_name)
    pdf_file_name = file_name.replace('.doc', '.pdf')
    reader = PdfReader(pdf_file_name)

    text = ''
    for page in reader.pages:
        text += page.extract_text().strip() + '\n'
    os.remove(pdf_file_name)
    for i in ['\u2006', '    ', '— ', '- ', '• ']: #
        text = text.replace(i, '')
    text = text.replace('\n', '|')
    return text

In [ ]:
def parse_doc(file_name):
    text = call((baseline_promt, get_text(file_name)))
    for i in ['```', '\n', '   ', 'python']:
        text = text.replace(i, '')
    text = text.replace('null', 'np.NaN')
    return eval(text)

In [ ]:
path = input('Input the path to where the resumes are located: ')
file_names = os.listdir(path)

vcs = []
errors = []
for i in tqdm(file_names):
    file_name = os.path.join(path,  i)
    try:
        vcs.append(parse_doc(file_name))
    except:
        errors.append(file_name)

In [ ]:
pd.DataFrame(vcs).to_csv('resumes_parsed.csv', index=False)